# <center>AMAL - TP8</center>
## <center>attention 机制</center>

### 简介（简要说明，参见课程）

在本次实验中，我们将研究**注意力机制**，这一机制使神经网络能够专注于特定的输入。例如，在判断一句话中是否存在某种情感时，无需关注所有的词语，只需关注那些表达情感的部分即可。

### 数据和模型的准备

您将使用 **IMDB** 数据集，该数据集包含来自互联网电影数据库（IMDb）的 50,000 条评论，以及 **GloVe** 词向量（请参见实验提供的代码）。目标是将一条评论分类为**正面**（pos）或**负面**（neg）。性能指标是分类的准确率。

### 1. 基础模型

- #### 问题 1

您将实现一个基于词嵌入的简单模型（见图 1）：一个文本 $ t = (t_1, \ldots, t_n) $ 用其包含的单词嵌入 $ \mathbf{x}_i $ 的平均值表示，即：  
$$
\hat{\mathbf{t}} = \frac{1}{n} \sum_i \mathbf{x}_i
$$  
请使用一个简单的线性模型，并采用交叉熵损失函数。

### 简单注意力机制

前面的模型效果不佳，因为所有单词在计算文本的质心表示时被赋予了相同的权重。此处，我们引入一个非常简单的注意力模型（见图 2），其目标是对与分类任务相关的重要单词赋予更高的权重，从而构建加权的文本表示：  
$$
\hat{t} = \sum_i p(a_i \mid t) x_i
$$  
其中，系数 $p(a_i \mid t)$ 表示关注文本中第 $i$ 个词的概率。随后，仍使用线性分类器进行分类。

我们建议将这些概率建模为：  
$$
\log p(a_i \mid t) = \text{常数} + q \cdot x_i
$$  
其中，$q$ 是一个被称为“问题”（或查询，**query**）的向量，它用于衡量第 $i$ 个词相对于所提问题的相关性。这个向量将在训练过程中学习（它是问题的嵌入向量）。

- #### 问题 2

1. **实现模型**。思考一种简单函数，能够根据 $q \cdot x_i$ 得到 $p(a_i \mid t)$。  
2. **训练过程中**，计算注意力的熵直方图。  
3. **推理时**，观察具体示例中模型关注的单词，从而了解哪些单词在决策中起了关键作用。

### 词汇点与全局注意力

词嵌入可以视为**键** $\textbf{k}_i$（英文为 *key*），我们希望找到与所提问题 $\textbf{q}$ 最相似的键。每个键都关联一个**值**（在我们的情况下，键和值是相同的，即单词的嵌入）。该值根据问题与键之间的相似度进行加权，即对第 $i$ 个值的注意力。通常，我们用向量 $\alpha(\textbf{q}, \textbf{k})$ 表示注意力权重。

### 3 问题与值

我们可能希望让问题 $\textbf{q}$ 依赖于文本本身。例如，表达对恐怖片感受的词汇可能与表达对浪漫片的词汇不同（尽管也有可能相似）。

- #### 问题 3

为了实现这一目标，您将根据文本计算问题 $q$，使用文本的平均表示 $\hat{t}_{m}$（见练习 1）并通过线性变换获得 $q$。（您可以根据实验结果自由地使模型更加复杂。）

然后，使用计算出的 $q$ 来表示文档，计算公式如下：  
$$
\log p(a_{i} \mid t) = \text{常数} + q(\hat{t}_{m}) \cdot x_{i}
$$  

同样，您可以通过改变单词嵌入的值来调整单词在决策中的表示：  
$$
\hat{\mathbf{t}} = \sum_i p(a_i \mid \mathbf{t}) v_\theta(\mathbf{x}_i)
$$  
其中，$v_\theta(\mathbf{x}_i)$ 是针对每个单词嵌入的可学习变换。

### 4 可选：上下文模型与注意力机制

- #### 问题 4

创建一个新模型，结合 **LSTM** 或 **GRU** 来生成上下文嵌入。您可以使用一层或多层 **LSTM/GRU**。这次，注意力机制应用于最后一层循环网络生成的状态序列。

与之前的问题类似，使用一个线性层进行分类，该线性层的输入为这些状态的加权和。

### 5 可选：熵

- #### 问题 5

为了强制注意力聚焦在输入的某一部分，可以引入**熵约束**。请修改注意力模型，加入这一正则化项，并观察其效果。